In [12]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [13]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:08<00:00, 1138.28it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



{'basic': {'expected_sample_burst_size': 2.990772263035298,
           'expected_sample_data_size': 16.53592546138685,
           'expected_time_to_recover': 99.769,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16037045575278894,
             'bit_redundancy': 418.845,
             'ideal_bit_efficency': 0.18937587864222816,
             'ideal_bit_redundancy': 342.4402842303757,
             'ideal_packet_efficiency': 0.9468793932111408,
             'ideal_packet_redundancy': 68.48805684607514,
             'ideal_time_to_recover': 84.48805684607514,
             'ideal_transmitted_bits': 422.4402842303757,
             'packet_efficiency': 0.8018522787639447,
             'packet_redundancy': 83.769,
             'permeability': 0.8468367613795381,
             'saturation': 16.0,
             'transmitted_bits': 498.845}}
Found implementation: anfext2
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:05<00:00, 1968.52it/s]


{'basic': {'expected_sample_burst_size': 2.993613539296691,
           'expected_sample_data_size': 16.553386819941725,
           'expected_time_to_recover': 39.683499999999995,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.4031902427961244,
             'bit_redundancy': 118.41749999999996,
             'ideal_bit_efficency': 0.47610557926930713,
             'ideal_bit_redundancy': 88.02995697462376,
             'ideal_packet_efficiency': 2.3805278963465355,
             'ideal_packet_redundancy': 17.605991394924757,
             'ideal_time_to_recover': 33.60599139492476,
             'ideal_transmitted_bits': 168.02995697462376,
             'packet_efficiency': 2.015951213980622,
             'packet_redundancy': 23.683499999999995,
             'permeability': 0.8468504893702612,
             'saturation': 16.0,
             'transmitted_bits': 198.41749999999996}}
Found implementation: anfext2l
  First time seen. Benchmarking.


...

{'basic': {'expected_sample_burst_size': 2.9973896318006963,
           'expected_sample_data_size': 16.55147462905294,
           'expected_time_to_recover': 32.3768,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.15443156828346222,
             'bit_redundancy': 136.88400000000001,
             'ideal_bit_efficency': 0.18239835625672557,
             'ideal_bit_redundancy': 112.062638683062,
             'ideal_packet_efficiency': 0.9119917812836278,
             'ideal_packet_redundancy': 22.412527736612404,
             'ideal_time_to_recover': 27.412527736612404,
             'ideal_transmitted_bits': 137.062638683062,
             'packet_efficiency': 0.7721578414173111,
             'packet_redundancy': 27.376800000000003,
             'permeability': 0.8466719297957921,
             'saturation': 5.0,
             'transmitted_bits': 161.88400000000001}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3449.15it/s]


{'basic': {'expected_sample_burst_size': 2.997924980047885,
           'expected_sample_data_size': 16.565522745411013,
           'expected_time_to_recover': 13.2441,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.35115114396258185,
             'bit_redundancy': 42.96709567122585,
             'ideal_bit_efficency': 0.4147002877135354,
             'ideal_bit_redundancy': 32.81939093496346,
             'ideal_packet_efficiency': 2.073501438567677,
             'ideal_packet_redundancy': 6.5638781869926905,
             'ideal_time_to_recover': 11.214559052747521,
             'ideal_transmitted_bits': 56.07279526373761,
             'packet_efficiency': 1.7557557198129095,
             'packet_redundancy': 8.593419134245169,
             'permeability': 0.8467588626443112,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.2205}}
Found implementation: single_binary_fullrank2
  First time see

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3502.04it/s]


{'basic': {'expected_sample_burst_size': 2.982981081295021,
           'expected_sample_data_size': 16.534710678382016,
           'expected_time_to_recover': 13.772099999999998,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.33768857804945007,
             'bit_redundancy': 45.607095671225835,
             'ideal_bit_efficency': 0.3986100334824698,
             'ideal_bit_redundancy': 35.08281998956528,
             'ideal_packet_efficiency': 1.993050167412349,
             'ideal_packet_redundancy': 7.016563997913057,
             'ideal_time_to_recover': 11.667244863667888,
             'ideal_transmitted_bits': 58.33622431833943,
             'packet_efficiency': 1.6884428902472504,
             'packet_redundancy': 9.121419134245167,
             'permeability': 0.8471652735361992,
             'saturation': 4.650680865754831,
             'transmitted_bits': 68.86049999999999}}
